In [4]:
"""
    製作一個客製menu

"""


#做一個menu的基本框架

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 843
  },
  "selected": true,
  "name": "自定義菜單-2",
  "chatBarText": "功能選單",
  "areas": [
    {
      "bounds": {
        "x": 66,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=sa"
      }
    },
    {
      "bounds": {
        "x": 676,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=develop"
      }
    },
    {
      "bounds": {
        "x": 1296,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type": "postback",
        "data": "type=question&question_type=sysops"
      }
    },
    {
      "bounds": {
        "x": 1906,
        "y": 155,
        "width": 510,
        "height": 510
      },
      "action": {
        "type":"uri",
        "uri":"https://github.com/iii-cutting-edge-tech-lab/Chatbot_Project"
      }
    }   
  ]
}
"""

In [6]:
"""
    user_id：就是姓名
    access_token：就像是ssh的私鑰
    secret_key：就是身份證字號，證明是line本人

"""

#擷取檔案內的變數，下面會使用

import json
secretFileContentJson=json.load(open("./secret_key",'r'))

access_token = secretFileContentJson.get("channel_access_token")
sercret_key = secretFileContentJson.get("secret_key")
user_id = secretFileContentJson.get("self_user_id")

In [5]:
"""
    將上面做好的menu框架post給line
    line會製作一個menu_id回傳過來
    之後就都是靠menu_id來連動menu
    一個bot最多一千個menu

"""


#使用requests來傳送封包

import requests

#讀上面的menu框架
menuJson=json.loads(menuRawData)

#line的位置
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
#http的header靠Authorization來辨識你是誰
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % access_token }
#使用post的方式新增menu進line，會回傳一個menu_id
lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

#看是否正確回覆
print(lineCreateMenuResponse)
#取出menu_id
print(lineCreateMenuResponse.text)





<Response [200]>
{"richMenuId":"richmenu-a9085916f4dd7237bfe8df1630cdbedb"}


In [10]:
'''
    得到menu_id後
    再對menu進行更多的設定

'''
import requests

# 從secret_key取得菜單Id 
uploadRichMenuId=secretFileContentJson.get("rich_menu_id")
print(uploadRichMenuId)

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 設定消息的基本安全憑證
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 上傳照片，這個照片檔要特別注意檔案格式，要是2500X843跟上面製作的rich_menu格式相符
uploadImageFile=open("./images/menu_2.jpg",'rb')
# 用request傳post http給line對menu_id做設定
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

# 檢查是否成功
print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-a9085916f4dd7237bfe8df1630cdbedb
https://api.line.me/v2/bot/richmenu/richmenu-a9085916f4dd7237bfe8df1630cdbedb/content
<Response [400]>
{"message":"An image has already been uploaded to the richmenu"}


In [12]:
"""

    將做好的menu_id跟menu資料
    post 到 API server
    
"""

import requests
#做一個將已經做好的menu進資料庫的動作
#網址
url = "http://192.168.122.100:5001/menu"
#表頭
Header={'Content-Type':'application/json'}
#傳送的資料，裝menu的格式
payload = {
  "menu_id":"richmenu-a9085916f4dd7237bfe8df1630cdbedb",
  "menu_content": menuRawData
}
#傳送封包
Response=requests.post(url,headers=Header,data=json.dumps(payload))
#檢驗
print(Response)
print(Response.text)

<Response [200]>
{
  "status_describe": "success add menu"
}



In [11]:
"""
    
    查詢line@綁定的menu

"""

# 取出用戶id，設定Line的遠端位置
user_id = 'Ua5ec88edae803461e0c943ef27a0312e'
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (user_id)
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)

https://api.line.me/v2/bot/user/Ua5ec88edae803461e0c943ef27a0312e/richmenu
<Response [404]>
{"message":"the user has no richmenu","details":[]}


In [9]:
"""

    刪除綁定在line@身上的menu

"""

#刪除原有榜定的menu
# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)


https://api.line.me/v2/bot/user/Ua5ec88edae803461e0c943ef27a0312e/richmenu
<Response [200]>
{}
